In [38]:
from langchain_core.tools import tool

@tool
def get_recent_orders_by_status(status, limit=10):
    """
    Retrieve a list of recent orders with the specified status.

    Args:
        status (str): The status of the orders to retrieve.
        limit (int): The maximum number of orders to retrieve (default is 10).

    Returns:
        str: A message confirming the function's execution.
    """
    # 여기서는 실제 데이터베이스에 접근하지 않고, 함수 동작 여부만 확인할 수 있도록
    # 단순한 문자열 메시지를 반환합니다.
    return f"Retrieving recent orders with status '{status}' up to {limit} records."

@tool
def create_order(user, product, quantity):
    """
    Place an order for a product with the specified quantity.

    Args:
        user (User): The user placing the order.
        product (Product): The product to order.
        quantity (int): The quantity of the product to order.

    Returns:
        str: A message confirming that the order has been placed.
    """
    # 여기에 주문을 처리하는 로직을 추가하세요.
    return f"Order placed for {quantity} {product.product_name}(s). Order completed."

@tool
def modify_order(order, new_quantity):
    """
    Modify an existing order to a new quantity.

    Args:
        order (Order): The order to modify.
        new_quantity (int): The new quantity for the order.

    Returns:
        str: A message confirming that the order has been modified.
    """
    # 여기에 주문 변경을 처리하는 로직을 추가하세요.
    return f"Order {order.id} modified to {new_quantity}. Order modification completed."

@tool
def cancel_order(order):
    """
    Cancel an existing order.

    Args:
        order (Order): The order to cancel.

    Returns:
        str: A message confirming that the order has been canceled.
    """
    # 여기에 주문 취소를 처리하는 로직을 추가하세요.
    return f"Order {order.id} canceled. Order cancellation completed."

In [39]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            너는 주문봇이야.
            사용자 입력에 대해 친절하게 답변해줘.
            사용자 입력에서 도구 호출에 필요한 인자를 모두 추출할 수 있을 때만 도구를 사용해야 해.
            모르는 내용을 너의 마음대로 지어내서는 절대로 안 돼. 모르는 내용은 모르겠다고 답해. 
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

In [40]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [41]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [53]:
tools = [get_recent_orders_by_status, create_order, modify_order, cancel_order]

In [54]:
from langchain_core.messages import AIMessage
from langchain_core.runnables import (
    Runnable,
)

def determine_tool_usage(msg: AIMessage) -> Runnable:
    """Determine whether to use the tool based on the given condition."""
    print(msg)
    if msg.additional_kwargs: 
        tool_map = {tool.name: tool for tool in tools}
        tool_calls = msg.tool_calls.copy()
        for tool_call in tool_calls:
            tool_call["output"] = tool_map[tool_call["name"]].invoke(tool_call["args"])
        print("도구 출력 결과: ", tool_call["output"])
        return str(tool_call["output"]) # 테스트
    else:
        return msg.content

In [55]:
chain_with_tools = prompt | model.bind_tools(tools) | determine_tool_usage

In [56]:
from langchain_core.runnables.history import RunnableWithMessageHistory

chain_with_tools_n_history  = RunnableWithMessageHistory(
    chain_with_tools,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [57]:
response = chain_with_tools_n_history.invoke(
                {"input": "주문 상태가 '주문 완료'인 주문을 알려줘"},
                config={"configurable": {"session_id": "test_240513-4"}}
            )
response 

Parent run 53e5e71c-540f-4c84-9173-fda67ff04355 not found for run 1a869220-819a-4fca-8499-39336ca33d92. Treating as a root run.


content="주문 상태가 '주문 완료'인 주문을 알려드리려면 주문 도구를 사용해야 합니다. 주문 도구를 사용하려면 주문 상태를 확인할 주문 번호가 필요합니다. 주문 번호를 알려주시면 주문 상태를 확인해 드릴게요." response_metadata={'token_usage': {'completion_tokens': 91, 'prompt_tokens': 798, 'total_tokens': 889}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-779f2da0-b274-4608-9b62-2935ea432f64-0'


"주문 상태가 '주문 완료'인 주문을 알려드리려면 주문 도구를 사용해야 합니다. 주문 도구를 사용하려면 주문 상태를 확인할 주문 번호가 필요합니다. 주문 번호를 알려주시면 주문 상태를 확인해 드릴게요."

In [52]:
response = chain_with_tools_n_history.invoke(
                {"input": "너가 사용 가능한 도구는 뭐야?"},
                config={"configurable": {"session_id": "test_240513-4"}}
            )
response 

Parent run 0d1f9ce8-b0e3-49a5-910b-2f2e06d8c520 not found for run 50592f10-29e9-4f96-803f-640682481efd. Treating as a root run.


content='제가 사용할 수 있는 도구는 주문 생성, 주문 수정, 주문 취소 등의 기능이 있어요. 필요하신 도구가 있으시면 언제든지 말씀해주세요.' response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 609, 'total_tokens': 667}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-df2e1a32-a3b0-439e-bbc8-2d63acfa07a4-0'


'제가 사용할 수 있는 도구는 주문 생성, 주문 수정, 주문 취소 등의 기능이 있어요. 필요하신 도구가 있으시면 언제든지 말씀해주세요.'

In [58]:
store["test_240513-4"].messages


[HumanMessage(content="나는 '나들'이야"),
 AIMessage(content="안녕하세요! '나들'님이세요. 무엇을 도와드릴까요?"),
 HumanMessage(content='주문 완료인 주문을 알려줘'),
 AIMessage(content='주문 상태를 알려드리려면 주문 번호나 상세 정보가 필요해요. 주문 번호나 주문 상세를 알려주시면 주문 상태를 확인해 드릴게요.'),
 HumanMessage(content="주문 상태가 '주문 완료'인 주문을 알려줘"),
 AIMessage(content='죄송해요, 제가 주문 상태를 확인하려면 주문 번호나 상세 정보가 필요해요. 주문 번호나 주문 상세를 제공해주시면 주문 상태를 확인할 수 있어요.'),
 HumanMessage(content='너가 사용 가능한 도구는 뭐야?'),
 AIMessage(content='제가 사용할 수 있는 도구는 주문 생성, 주문 수정, 주문 취소 등의 기능이 있어요. 필요하신 도구가 있으시면 언제든지 말씀해주세요.'),
 HumanMessage(content="주문 상태가 '주문 완료'인 주문을 알려줘"),
 AIMessage(content="주문 상태가 '주문 완료'인 주문을 알려드리려면 주문 도구를 사용해야 합니다. 주문 도구를 사용하려면 주문 상태를 확인할 주문 번호가 필요합니다. 주문 번호를 알려주시면 주문 상태를 확인해 드릴게요.")]

In [61]:
response = chain_with_tools_n_history.invoke(
                {"input": "2번 주문을 취소하고 싶어"},
                config={"configurable": {"session_id": "test_240513-5"}}
            )
response 

Parent run ffc368da-d3f1-417c-b52a-dd4a2bbf0efd not found for run 69d976ae-bd25-43a9-ba4d-d91282eedf5a. Treating as a root run.


content='취소할 주문의 정보가 필요해요. 취소하고 싶은 주문이 어떤 제품인지 알려주실 수 있나요?' response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 535, 'total_tokens': 584}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-40ea135f-c8a2-4c45-adf7-39f7ebb9a7de-0'


'취소할 주문의 정보가 필요해요. 취소하고 싶은 주문이 어떤 제품인지 알려주실 수 있나요?'

In [63]:
from django.db import connection

def get_recent_orders_by_status(user_id, status):
    """
    Retrieve the three most recent orders with a specific status for the given user.
    """
    with connection.cursor() as cursor:
        cursor.execute(
            """
            SELECT 
                o.id, 
                o.user_id, 
                o.created_at, 
                o.order_status,
                oi.product_name,
                oi.quantity,
                oi.price
            FROM 
                orders_order o
            JOIN 
                products_orderitem oi ON o.id = oi.order_id
            JOIN 
                orders_orderstatus os ON o.id = os.order_id
            WHERE 
                o.user_id = %s
                AND os.status = %s
            ORDER BY 
                o.created_at DESC
            LIMIT 3
            """,
            [user_id, status]
        )
        recent_orders = cursor.fetchall()
    return recent_orders


In [64]:
tools = [get_recent_orders_by_status]

In [65]:
chain_with_tools = prompt | model.bind_tools(tools) | determine_tool_usage

In [66]:
chain_with_tools_n_history  = RunnableWithMessageHistory(
    chain_with_tools,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [68]:
response = chain_with_tools_n_history.invoke(
                {"input": "주문 상태가 'order'인 주문을 알고 싶어"},
                config={"configurable": {"session_id": "test_240513-6"}}
            )
response 

Parent run 6574eb0e-f4ba-4f25-b626-cbef22b1fe77 not found for run b5e3bebb-8e76-4738-a898-24586b96eade. Treating as a root run.


content='' additional_kwargs={'tool_calls': [{'id': 'call_yZ4GbSZnGMzEoiACY1nN1eFk', 'function': {'arguments': '{}', 'name': 'get_recent_orders_by_status'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 234, 'total_tokens': 247}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None} id='run-9f947f8a-8c03-41cb-b8d5-5ad4307107c0-0' tool_calls=[{'name': 'get_recent_orders_by_status', 'args': {}, 'id': 'call_yZ4GbSZnGMzEoiACY1nN1eFk'}]


AttributeError: 'function' object has no attribute 'name'